# Importing the necessary library to process the Audio files and extract the features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import librosa as lr
import re
from IPython.display import Audio
from scipy.fftpack import fft, rfft
from scipy import stats
from librosa.core import piptrack
from scipy.stats.mstats import gmean

# Appending the directories for the audio files in a list, to open them later.

In [2]:
audio_files = glob('..//audios_to_test//friends_voices//*.wav')

In [3]:
len(audio_files)

8

In [4]:
audio_files[1]

'..//audios_to_test//friends_voices\\f2 - Maria.wav'

## Opening the audio files using librosa, the autputs are:
- audio = audio time series
- sampling_rate = sampling rate of audio

In [5]:
# TO open an audio file at a time and listen to it in the notebook

audio, sampling_rate = lr.load(audio_files[5])
Audio(audio, rate=sampling_rate)

## To plot each audio file in the time domain

In [6]:
# for file in range(0, len(audio_files), 1):
#     audio, sampling_rate = lr.load(audio_files[file])
#     time = np.arange(0, len(audio)) / sampling_rate
#     plt.plot(time, audio)
#     plt.xlabel('Time (secs)')
#     plt.show()

# Creating a function to extract the features needed out of an Audio file and then append the features into a dictionary

In [7]:
def feature_extractor(audio: np.ndarray, sampling_rate: int) -> dict:
    frequency_spectrum = np.abs(np.fft.rfft(audio))
    frequency = np.fft.rfftfreq(len(audio), d=1 / sampling_rate)
    frequency_spectrum = np.abs(frequency_spectrum)
    amplitude = frequency_spectrum / frequency_spectrum.sum()
    mean_frequency = (frequency * amplitude).sum()
    freq_standerd_deviation = np.sqrt(np.sum(amplitude * ((frequency - mean_frequency) ** 2)))
    amplitude_cumulative_sum = np.cumsum(amplitude)
    median_frequency = frequency[len(amplitude_cumulative_sum[amplitude_cumulative_sum <= 0.5]) + 1]
    mode_frequency = frequency[amplitude.argmax()]
    quartile_25 = frequency[len(amplitude_cumulative_sum[amplitude_cumulative_sum <= 0.25]) + 1]
    quartile_75 = frequency[len(amplitude_cumulative_sum[amplitude_cumulative_sum <= 0.75]) + 1]
    interquartile_range = quartile_75 - quartile_25
    deviation_from_mean = amplitude - amplitude.mean()
    amplitude_std = amplitude.std()
    skewness = ((deviation_from_mean ** 3).sum() / (len(frequency_spectrum) - 1)) / amplitude_std ** 3
    kurtosis = ((deviation_from_mean ** 4).sum() / (len(frequency_spectrum) - 1)) / amplitude_std ** 4
    centroid_frequency = lr.feature.spectral_centroid(y=audio, sr=sampling_rate)
    spectral_flatness = lr.feature.spectral_flatness(y=audio)
    pitches, magnitudes = piptrack(y=audio, sr=sampling_rate, fmax = 280)

    dictionary_of_features = {
        'Mean_freq': mean_frequency/1000,
        'Std': freq_standerd_deviation/1000,
        'Median_freq': median_frequency/1000,
        'Mode_freq': mode_frequency/1000,
        'First_quartile': quartile_25/1000,
        'Third_quartile': quartile_75/1000,
        'Interquantile_range': interquartile_range/1000,
        'Skewness': skewness,
        'Kurtosis': kurtosis,
        'Centroid_freq': np.mean(centroid_frequency)/1000,
        'Spectral_flatness_measure': np.mean(spectral_flatness),
        'Mean_fundamental_freq': (pitches[np.nonzero(pitches)].mean())/1000,
        'Min_fundamental_freq': (pitches[np.nonzero(pitches)].min())/1000,
        'Max_fundamental_freq': (pitches[np.nonzero(pitches)].max())/1000
    }

    return dictionary_of_features

# To open all the audio files, plot the frequency domain of ech file and apply the feature_extractor function to pull the features from the audio files and then append them into a list to use in creating a dataframe later.

### I added a condition to the loop to use the file name and append a new key: values pair in the dictionary, if the file starts with 'f' it appends 1 to a Gender key in the dictionary, if the file starts with anything else 'm', it appends 0 to the Gender key in the dictionary. 

In [8]:
list_of_dict = []
for file in range(0, len(audio_files), 1):
    audio, sampling_rate = lr.load(audio_files[file])
    frequencies = rfft(audio)
    #plt.plot(abs(frequencies))
    #plt.show()
    #print(audio_files[file])
    dictionary_of_features = feature_extractor(audio, sampling_rate)
    if audio_files[file].startswith('..//audios_to_test//friends_voices\\f'):
        dictionary_of_features['Gender'] = 1
    elif  audio_files[file].startswith('..//audios_to_test//friends_voices\\m'):
        dictionary_of_features['Gender'] = 0
    list_of_dict.append(dictionary_of_features)

## Printing the list of dictionaries to make sure that all the dictionaries are in it.

In [9]:
#list_of_dict

## Creating a dataframe for the features extracted from the audio files

In [10]:
friends_voices = pd.DataFrame(list_of_dict)

In [11]:
friends_voices.head()

,Centroid_freq,First_quartile,Gender,Interquantile_range,Kurtosis,Max_fundamental_freq,Mean_freq,Mean_fundamental_freq,Median_freq,Min_fundamental_freq,Mode_freq,Skewness,Spectral_flatness_measure,Std,Third_quartile
0,1.919026,0.441396,1,2.676911,64.671573,0.284953,2.253995,0.208517,0.868276,0.146402,0.370469,6.671048,0.018473,2.717403,3.118307
1,1.879197,0.474954,1,1.842071,52.318483,0.284990,1.767053,0.218724,0.922768,0.145374,0.477841,5.921802,0.020872,1.975538,2.317026
2,2.158872,0.535558,1,2.614442,130.643402,0.284909,2.114163,0.215908,1.312398,0.146029,0.205835,8.977198,0.021229,2.158895,3.150000
3,1.970617,0.473727,0,1.467732,76.364557,0.285286,1.658160,0.218546,0.743681,0.145933,0.486627,7.224239,0.032685,2.017657,1.941459
4,2.163452,0.603609,0,2.042229,145.239398,0.285239,2.186584,0.215124,1.477669,0.145533,0.245252,7.634636,0.044122,2.206759,2.645838


In [13]:
friends_voices.to_csv('..//datasets//friends_voices.csv', index=False)